#Digital Ration Card System Prompt Project

###Idea: Create a digital ration card system that allows users to check their entitlements and find nearby ration shops.
###Use cases: - A family checks their remaining food grain quota for the month.A ration shop owner verifies a customer’s entitlement.

In [ ]:


import pandas as pd
from google.colab import files


df = pd.DataFrame({

    'family_id': ['FAM1', 'FAM2', 'FAM3', 'FAM4', 'FAM5'],

    'food_grain_quota': [50, 75, 100, 125, 150],

    'ration_shop_id': ['SHOP1', 'SHOP2', 'SHOP3', 'SHOP4', 'SHOP5'],

    'location': ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai']

})


df2 = pd.DataFrame({

    'shop_id': ['SHOP1', 'SHOP2', 'SHOP3', 'SHOP4', 'SHOP5'],

    'location': ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai'],

    'address': ['123 Main St', '456 Market St', '789 High St', '901 Low St', '234 Broad St'],

    'phone_number': ['1234567890', '9876543210', '5551234567', '7654321098', '3421098765']

})



df.to_csv('ration_cards.csv', index=False)
df2.to_csv('ration_shops.csv', index=False)


files.download('ration_cards.csv')
files.download('ration_shops.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
import groq
print(groq.__version__)

0.11.0


In [ ]:
from groq import Groq

In [ ]:
import os
import requests
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq

# Set Groq API key
GROQ_API_KEY = "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"

# Load the dataset
ration_cards_df = pd.read_csv('/content/ration_cards.csv')
ration_shops_df = pd.read_csv('/content/ration_shops.csv')

# Define a function to preprocess the data
def preprocess_data(ration_cards_df, ration_shops_df):
    # Convert the ration card data to lowercase
    ration_cards_df['family_id'] = ration_cards_df['family_id'].str.lower()
    ration_cards_df['food_grain_quota'] = ration_cards_df['food_grain_quota'].astype(str).str.lower()
    ration_cards_df['ration_shop_id'] = ration_cards_df['ration_shop_id'].str.lower()

    # Convert the ration shop data to lowercase
    ration_shops_df['shop_id'] = ration_shops_df['shop_id'].str.lower()
    ration_shops_df['location'] = ration_shops_df['location'].str.lower()

    return ration_cards_df, ration_shops_df

# Preprocess the data
ration_cards_df, ration_shops_df = preprocess_data(ration_cards_df, ration_shops_df)

# Define a function to find nearby ration shops
def find_nearby_ration_shops(user_input, ration_cards_df, ration_shops_df):
    # Use TF-IDF to convert the user input and ration shop locations to vectors
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer to both user input and ration shop locations
    all_text = [user_input] + ration_shops_df['location'].tolist()
    vectorizer.fit(all_text)

    # Transform user input and ration shop locations into vectors
    user_input_vector = vectorizer.transform([user_input])
    ration_shop_vectors = vectorizer.transform(ration_shops_df['location'])

    # Use cosine similarity to find the most similar ration shops
    similarities = cosine_similarity(user_input_vector, ration_shop_vectors)

    # Get the index of the most similar ration shop
    most_similar_index = np.argmax(similarities)

    # Get the most similar ration shop
    most_similar_shop = ration_shops_df.iloc[[most_similar_index]]

    return most_similar_shop

# Define a function to generate output using LLM
def generate_output(most_similar_shop, api_key):
    # Use Groq API to generate output
    client = Groq(api_key=api_key)

    # Use client.chat.completions.create() to interact with Groq models
    prompt = f"Generate a description for the nearby ration shop: {most_similar_shop['shop_id'].tolist()}"
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        model="llama3-8b-8192",  # or other appropriate model
    )
    output = response.choices[0].message.content  # Extract the generated text from the response

    return output

# CLI function
def cli():
    print("Welcome to the Digital Ration Card System!")
    while True:
        user_input = input("Enter a prompt (e.g., 'Check my ration card quota' or 'Find nearby ration shops'): ")
        if user_input.startswith("Check my ration card quota"):
            family_id = input("Enter your family ID: ")
            # Check if the family_id exists in the DataFrame before accessing quota
            if family_id in ration_cards_df['family_id'].values:
                quota = ration_cards_df[ration_cards_df['family_id'] == family_id]['food_grain_quota'].values[0]
                print(f"Your remaining food grain quota is {quota} kg.")
            else:
                print(f"Family ID {family_id} not found in the database.")  # Handle the case where family_id is not found
        elif user_input.startswith("Find nearby ration shops"):
            location = input("Enter your location: ")
            most_similar_shop = find_nearby_ration_shops(location, ration_cards_df, ration_shops_df)
            output = generate_output(most_similar_shop, GROQ_API_KEY)
            print("Response:")
            print(output)

        else:
            print("Invalid prompt. Please try again!")
# Run the CLI
if __name__ == "__main__":
    cli()

Welcome to the Digital Ration Card System!
Enter a prompt (e.g., 'Check my ration card quota' or 'Find nearby ration shops'): Check my ration card quota
Enter your family ID: FAM2
Family ID FAM2 not found in the database.
Enter a prompt (e.g., 'Check my ration card quota' or 'Find nearby ration shops'): Find nearby ration shops
Enter your location: Delhi
Response:
**Shop2: Your One-Stop Shop for Daily Essentials**

Located just around the corner, Shop2 is a convenient and friendly ration shop that offers a wide range of daily essentials at affordable prices. With a dedicated team of professionals, this shop is committed to providing excellent customer service and ensuring that you get the best value for your money.

**Products and Services**

* Groceries: Fresh fruits and vegetables, packaged foods, and staples like rice, wheat flour, and pulses.
* Beverages: Soft drinks, packaged water, and other daily beverages.
* Personal Care: Soaps, detergents, toothpaste, and other essential pers